# Architectures used for experiments: 

## EffecientNet

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torchvision.transforms import transforms

class EfficientNet(nn.Module):
    def __init__(self, num_classes, hidden_size=256, num_layers=2, dropout=0.25):
        super().__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        for param in self.efficientnet.parameters():
            param.requires_grad = False
        self.lstm = nn.LSTM(input_size=1280, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.dropout1 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size * 2, 256)
        self.dropout2 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(256, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.efficientnet.extract_features(x)
        # Flatten the feature maps and add a batch dimension
        x = x.flatten(start_dim=2).permute(0, 2, 1)
        # Pass the flattened feature maps through the bidirectional LSTM layer
        x, _ = self.lstm(x)
        # Concatenate the outputs of the forward and backward directions and pass through the fully connected layers with dropout
        x = self.dropout1(torch.cat((x[:, -1, :self.lstm.hidden_size], x[:, 0, self.lstm.hidden_size:]), dim=1))
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


## ResNet18

In [2]:
class ResNet(nn.Module):
    def __init__(self, num_classes, hidden_size=512, num_layers=2, bidirectional=True, dropout=0.5):
        super(ResNetWithLSTM, self).__init__()

        # Load pre-trained ResNet-18 model
        self.resnet = models.resnet18(pretrained=True)

        # Freeze all layers except for the last fully connected layer
        for param in self.resnet.parameters():
            param.requires_grad = False
        self.fc1 = nn.Linear(1000, 512)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)  
        self.lstm = nn.LSTM(input_size=hidden_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            bidirectional=bidirectional,
                            batch_first=True)
        self.dropout2 = nn.Dropout(dropout) 
        self.relu2 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size * 2 if bidirectional else hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)  # Apply dropout after the first fully connected layer
        x, _ = self.lstm(x)
        x = self.dropout2(x)  # Apply dropout after the LSTM layer
        x = self.relu2(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


## MobileNet_v2

In [ ]:
class MobileNet(nn.Module):
    def __init__(self, num_classes, hidden_size=256, num_layers=2, dropout=0.25):
        super().__init__()
        self.mobilenet = torchvision.models.mobilenet_v2(pretrained=True).features
        for param in self.mobilenet.parameters():
            param.requires_grad = False
            
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.lstm = nn.LSTM(input_size=1280, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.dropout1 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size * 2, 512)
        self.dropout2 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(512, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.mobilenet(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = x.unsqueeze(1)
        x = x.expand(-1, self.lstm.num_layers * 2, -1)
        x, _ = self.lstm(x)
        x = self.dropout1(torch.cat((x[:, -1, :self.lstm.hidden_size], x[:, 0, self.lstm.hidden_size:]), dim=1))
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x
